# Automated 2084
Using ```selenium```, create a function named play_2084that does the following:
* Starts an instance of Chrome
* Navigates to https://play2048.co/
* Plays 10 consecutive games of 2084 and indicates when it is starting each game
* Completes all games without human intervention
* Returns the best score for the 10 games

A hint is available, but will cost you 5 points. One of the difficult parts of this exercise is determining when the game is over. I have one way to do it that I will share with you for 5 points. If you want the hint and agree not to share it with others, contact the TA.

In [1]:
# Run to initialize, this code uses an algorithem (which when fails) calls chatgpt to make decision on best move for the board

! conda install -c conda-forge openai -y


Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /home/sam/anaconda3/envs/mis501

  added / updated specs:
    - openai


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openai-1.13.3              |     pyhd8ed1ab_0         122 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         122 KB

The following NEW packages will be INSTALLED:

  distro             conda-forge/noarch::distro-1.9.0-pyhd8ed1ab_0 
  h2                 conda-forge/noarch::h2-4.1.0-pyhd8ed1ab_0 
  hpack              conda-forge/noarch::hpack-4.0.0-pyh9f0ad1d_0 
  httpcore           conda-forge/noarch::httpcore-1.0.4-pyhd8ed1ab_0 
  httpx              conda-forge/noarch::httpx-0.27.0-pyhd8ed1ab_0 
  hyperframe         conda-forge/noarch::hyperframe-6.0.1-pyhd8

In [11]:
! conda install -c conda-forge selenium -y

Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from openai import OpenAI
import time, random
from selenium.common.exceptions import NoSuchElementException
import pprint



In [7]:

driver = webdriver.Chrome()
driver.get('https://2048game.com')





time.sleep(2)	# wait for page to load

The chromedriver version (121.0.6167.85) detected in PATH at /usr/bin/chromedriver might not be compatible with the detected chrome version (122.0.6261.94); currently, chromedriver 122.0.6261.94 is recommended for chrome 122.*, so it is advised to delete the driver in PATH and retry


In [9]:

gameElem = driver.find_element(By.CLASS_NAME,'grid-container')

directions = {'up':Keys.UP, 'down':Keys.DOWN, 'right':Keys.RIGHT, 'left':Keys.LEFT}
direction = [Keys.UP,Keys.DOWN,Keys.RIGHT,Keys.LEFT]
ActionChains(driver).move_to_element_with_offset(gameElem, 0, 50).perform()

time.sleep(3)


##NOTE: I USED CHATGPT TO RE-WRITE THIS FUNCITON TO RUN AS FAST AS POSSIBLE FOR TIME SAKE
## MY ORIGINAL FUNCITON IS COMMENTED OUT BELOW, WANTED TO SAVE YOU GUYS AS MUCH TIME AS POSSIBLE FOR RUNNING IT
def print_current_tiles():
    time.sleep(.15)
    grid = [['0' for _ in range(4)] for _ in range(4)]  # Initialize the grid with '0's
    tiles = driver.find_elements(By.XPATH, "//*[contains(@class, 'tile-position-')]")

    for tile in tiles:
        classes = tile.get_attribute('class').split()
        position_class = next((cls for cls in classes if 'tile-position-' in cls), None)
        
        if position_class:
            try:
                # Extract position (ignoring other parts of the class name if present)
                parts = position_class.split('-')
                x, y = int(parts[2]) - 1, int(parts[3]) - 1  # Convert to 0-based index
                
                # Find the tile value and assign it to the correct position in the grid
                value = tile.find_element(By.CLASS_NAME, "tile-inner").text
                grid[y][x] = value
            except (IndexError, ValueError) as e:
                # Handle cases where the class name is not as expected or conversion to int fails
                print(f"Error parsing tile position from class '{position_class}': {e}")
                continue  # Skip this tile and continue with the next

    return grid


## MY ORIGINAL FUNCTION
# def print_current_tiles():
#     time.sleep(.25)
#     grid = []
#     for i in range(1,5):
#         rows = []
#         for g in range(1,5):
#             try:
#                 cell = driver.find_element(By.CLASS_NAME, f"tile-position-{g}-{i}.tile-merged")
#                 rows.append(cell.find_element(By.CLASS_NAME, "tile-inner").text)
#             except NoSuchElementException:
#                 try:
#                     cell = driver.find_element(By.CLASS_NAME, f"tile-position-{g}-{i}")
#                     rows.append(cell.find_element(By.CLASS_NAME, "tile-inner").text)
#                 except:
#                     rows.append('0')
#         grid.append(rows)
#     #print(grid)
#     return grid    

def is_over(browser):
	try:
		game_over = browser.find_element(By.CLASS_NAME,'game-over')
		return game_over
	except:
		pass

def call_openai(grid, attempted_moves):
    client = OpenAI(api_key="YOUR OPENAI KEY")   
    grid_str = "\n".join(["[" + ", ".join(row) + "]" for row in grid])
    prompt = f"""Given the current state of a 2048 game board as follows:
        {grid_str}
         with '0' representing an empty cell. Based on this row-by-column representation of the board, state the best possible move only (up, down, left, right). The player has already attempted these moves:{attempted_moves} thus they cannot be used again. The decision should aim to consolidate similar values to form higher-numbered tiles and to position the board advantageously for subsequent moves, while avoiding actions that could lead to a no-win situation. YOU MUST provide only one of the following directions as your response, no other text/explanation should be given: [up, down, left, or right].
         """
    message = {"role": "system",
    "content": prompt
    }
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[message]
    )
    return completion.choices[0].message.content
    
def count_empty_spaces(grid):
    return sum(row.count('0') for row in grid)

def best_move():
    move_order = ['up', 'left', 'down', 'right'] 
    return move_order


game_ended = False
i = 0
last_grid = None
attempted_moves = []
chats_attempted_moves = []
scores = []
while not game_ended:
    game_ended = is_over(driver)
    if game_ended:
        print('Game over!')
        i += 1
        print(f"Score for round {i}:",driver.find_element(By.CLASS_NAME, 'score-container').text)
        scores.append(driver.find_element(By.CLASS_NAME, 'score-container').text)
        if i == 10:  
            print("\n" * 15)
            print("Here are the scores!",scores)
            break
        time.sleep(1)
        restartElem = driver.find_element(By.CLASS_NAME, 'retry-button')
        restartElem.click()
        game_ended = False  
        last_grid = None
        attempted_moves.clear()
        chats_attempted_moves.clear()
    else:
        current_grid = print_current_tiles()  
        if count_empty_spaces(current_grid) <= 5:
            if last_grid and current_grid == last_grid:
                if choice_str:
                    chats_attempted_moves.append(choice_str.strip().lower())
            else:
                chats_attempted_moves.clear()       
            choice_str = call_openai(current_grid, chats_attempted_moves)
            #print("ChatGPT:", choice_str)
            choice = directions.get(choice_str.strip().lower(), None)
        else:
            if last_grid is not None and last_grid == current_grid:
                move_order = [m for m in best_move() if m not in attempted_moves]
            else:
                move_order = best_move()
                attempted_moves.clear()
            if move_order:
                choice_str = move_order[0]  
                choice = directions[choice_str]
                #print("Best Choice:", choice_str)
                attempted_moves.append(choice_str) 

        ActionChains(driver).send_keys(choice).perform()
        last_grid = current_grid




Game over!
Score for round 1: 5120
Game over!
Score for round 2: 5120


KeyboardInterrupt: 